In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import selenium
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
import sklearn
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans


In [ ]:
url = 'https://www.jumbo.cl/frutas-y-verduras'

# Realizamos la solicitud HTTP para obtener el HTML
response = requests.get(url)
html = response.text

# Creamos el objeto BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

productos = soup.find_all('div', class_='product-card')

productos

In [ ]:
enlaces_productos = []
for producto in productos:
    # Buscar el anchor con la clase específica dentro de cada producto
    anchor = producto.find('a', class_='product-card-image-link not-logged')
    if anchor:
        enlace = anchor['href']  # Extraer el link del anchor
        enlaces_productos.append(enlace)

# 4. Mostrar o procesar los enlaces
for enlace in enlaces_productos:
    print(enlace)



In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

base_url = 'https://www.jumbo.cl/frutas-y-verduras?page='  # Asegúrate de que la URL base sea correcta
pagina = 1
enlaces_productos = []

while pagina <= 13:  # Terminar en la página 13
    # Concatenar el número de página a la URL base
    url = f'{base_url}{pagina}'
    print(f'Accediendo a: {url}')
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar los divs que contienen los productos
    productos = soup.find_all('div', class_='product-card')  # Ajustar la clase si es necesario
    
    # Si no se encuentran productos, romper el loop
    if not productos:
        print(f"No se encontraron productos en la página {pagina}. Terminando paginación.")
        break

    # Extraer los anchors dentro de cada tarjeta de producto
    for producto in productos:
        anchor = producto.find('a', class_='product-card-image-link not-logged')
        if anchor:
            enlace = anchor['href']
            enlaces_productos.append(enlace)
    
    # Ir a la siguiente página
    pagina += 1

# Mostrar o procesar los enlaces obtenidos
for enlace in enlaces_productos:
    print(enlace)

# Guardar los enlaces en un archivo CSV
with open('enlaces_frutasverduras.csv', mode='w', newline='') as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(['enlace'])  # Cabecera del archivo CSV
    for enlace in enlaces_productos:
        writer.writerow([enlace])


print("Enlaces guardados en 'enlaces_frutasverduras.csv'.")


In [2]:
import csv

# Leer el archivo CSV con los enlaces
enlaces = []
with open('enlaces_frutasverduras.csv', mode='r') as archivo_csv:
    reader = csv.reader(archivo_csv)
    next(reader)  # Saltar la cabecera
    for row in reader:
        enlaces.append(row[0])  # Cada enlace es una fila

print("Enlaces leídos:", len(enlaces))

Enlaces leídos: 448


In [ ]:
import json
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# Configuración de las opciones de Chrome
options = Options()
options.binary_location = r'C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe'
service = Service(r"C:\Users\SIMON\Desktop\Kaggle\scraping\chromedriver.exe")

# Inicializa el driver
driver = webdriver.Chrome(service=service, options=options)

# Leer enlaces desde el archivo CSV, ignorando el carácter ';'
with open('enlaces_frutasverduras.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter=',')  # Usa la coma como delimitador
    enlaces = [row[0].strip().replace(';', '') for row in reader if row]  # Limpiar espacios y eliminar ';'

# Limitar a los primeros 10 enlaces
enlaces = enlaces[:10]

# Lista para almacenar los datos nutricionales
datos_nutricionales = []

# Procesar cada enlace
for enlace in enlaces:
    url_producto = f'https://www.jumbo.cl{enlace}'  # Construir la URL del producto
    print(f'Accediendo a: {url_producto}')
    
    # Crear un diccionario para almacenar los nutrientes del producto
    producto_info = {
        'Marca': '',
        'Codigo Producto': '',
        'Nombre Producto': '',
        'Precio': '',
        'Nutrientes': [],  # Lista para almacenar los nutrientes
        'Ingredientes': ''  # Campo para almacenar los ingredientes
    }
    
    try:
        driver.get(url_producto)
        time.sleep(2)  # Espera a que la página cargue

        # Extraer el H1
        h1_element = driver.find_element(By.CSS_SELECTOR, 'h1.product-name.text-black.text-lg.font-bold.lg\\:text-2xl.lg\\:mb-0\\.5')
        marca = driver.find_element(By.CSS_SELECTOR, 'span.product-brand.leading-\\[18px\\].text-primary500.text-base.font-semibold.text-black-600.underline.capitalize')
        marca_producto = marca.text.strip()
        nombre_producto = h1_element.text.strip()
        id_product = driver.find_element(By.CSS_SELECTOR, 'span.product-code.text-greyMidDark.text-sm.lg\\:mb-2')
        codigo_producto = id_product.text.strip()  # Obtener el id
        precio = driver.find_element(By.CSS_SELECTOR, 'span.prices-main-price')
        precio = precio.text.strip()  # Obtener el precio del producto
        # Hacer clic en la pestaña de "Información nutricional"
        informacion_nutricional_tab = driver.find_element(By.XPATH, "//span[contains(text(), 'Información nutricional')]")
        driver.execute_script("arguments[0].scrollIntoView();", informacion_nutricional_tab)
        informacion_nutricional_tab.click()
        
        

        time.sleep(2)  # Espera a que se cargue la tabla

        # Encuentra todos los <li> dentro de los <ul> que tienen la clase 'nutritional-details-container-data'
        li_elements = driver.find_elements(By.CSS_SELECTOR, 'ul.nutritional-details-container-data li')

        # Verificar cuántos <li> se encontraron
        print(f'Total de <li> encontrados: {len(li_elements)}')

        # Actualizar el diccionario con la información del producto
        producto_info['Marca'] = marca_producto
        producto_info['Codigo Producto'] = codigo_producto
        producto_info['Nombre Producto'] = nombre_producto
        producto_info['Precio'] = precio

        # Procesar los elementos, asumiendo que la estructura es nutriente seguido de dos valores
        for i in range(0, len(li_elements), 3):  # Incrementar de 3 en 3
            if i + 2 < len(li_elements):  # Asegurarse de que hay suficiente para evitar errores de índice
                nutriente = li_elements[i].text.strip()  # Nombre del nutriente
                valor_por_100g = li_elements[i + 1].text.strip()  # Valor por cada 100g
                valor_por_porcion = li_elements[i + 2].text.strip()  # Valor por porción
                print(f'Nutriente: {nutriente}, Valor por 100g: {valor_por_100g}, Valor por porción: {valor_por_porcion}')
                
                # Añadir el nutriente a la lista dentro del diccionario del producto
                producto_info['Nutrientes'].append({
                    'Nutriente': nutriente,
                    'Valor por 100g': valor_por_100g,
                    'Valor por porcion': valor_por_porcion
                })
            else:
                print(f'Elemento sin suficientes valores encontrado: {li_elements[i].text.strip()}')

        # Añadir el diccionario del producto a la lista de datos
    except Exception as e:
        print(f"Error al procesar {url_producto}: {str(e)}")

    try:
        ingredientes_tab = driver.find_element(By.XPATH, "//span[contains(text(), 'Ingredientes')]")
        ingredientes_tab.click()
        time.sleep(1)  # Espera breve para asegurar que el contenido se carga

        # Extrae el contenido del div correspondiente a los ingredientes
        ingredientes_div = driver.find_element(By.CSS_SELECTOR, 'div.text-base.leading-5')
        ingredientes_texto = ingredientes_div.text.strip()

        # Agrega los ingredientes al diccionario del producto
        producto_info['Ingredientes'] = ingredientes_texto
        print(f'Ingredientes: {ingredientes_texto}')

    except:
        # Si no se encuentra la pestaña "Ingredientes", simplemente continúas sin agregarla
        print("No se encontró la pestaña de 'Ingredientes' para este producto.")
    datos_nutricionales.append(producto_info)
# Comprobar si se capturaron datos antes de guardar
if datos_nutricionales:
    # Guardar los datos en un archivo JSON
    nombre_archivo_json = 'datos_nutricionalesss.json'
    with open(nombre_archivo_json, 'w', encoding='utf-8') as file:
        json.dump(datos_nutricionales, file, ensure_ascii=False, indent=4)

    print(f"Datos guardados en {nombre_archivo_json}")

    # También puedes guardar en CSV si lo deseas
    # Ten en cuenta que el formato del CSV será diferente debido a la estructura
    nombre_archivo_csv = 'df_verduras_frutas.csv'
    with open(nombre_archivo_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Codigo Producto', 'Nombre Producto', 'Nutriente', 'Valor por 100g', 'Valor por porcion']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for producto in datos_nutricionales:
            for nutriente in producto['Nutrientes']:
                writer.writerow({
                    'Codigo Producto': producto['Codigo Producto'],
                    'Nombre Producto': producto['Nombre Producto'],
                    'Nutriente': nutriente['Nutriente'],
                    'Valor por 100g': nutriente['Valor por 100g'],
                    'Valor por porcion': nutriente['Valor por porcion']
                })

    print(f"Datos guardados en {nombre_archivo_csv}")
else:
    print("No se encontraron datos nutricionales.")

# Cerrar el driver después de terminar
driver.quit()

# Cargar y mostrar el DataFrame
df = pd.read_json('datos_nutricionalesss.json')


                                            

In [15]:
import pandas as pd
import json
import re

# Cargar el JSON en un DataFrame
with open('datos_nutricionalesss.json', 'r', encoding='latin1') as file:
    datos = json.load(file)


# Crear el DataFrame
df = pd.json_normalize(datos, 'Nutrientes', ['Marca','Codigo Producto','Nombre Producto', 'Precio', 'Ingredientes'], errors='ignore')

# Verificar si las columnas 'Valor por 100g' y 'Valor por porción' existen en el DataFrame
    # Pivotar el DataFrame para obtener formato ancho
df_ancho3 = df.pivot_table(index=['Nombre Producto', 'Precio', 'Codigo Producto', 'Marca', 'Ingredientes'], 
                               columns='Nutriente', 
                               values=['Valor por 100g', 'Valor por porcion'], 
                               aggfunc='first')

# Aplanar las columnas multiíndice
df_ancho3.columns = [f'{nutriente} {valor}' for valor, nutriente in df_ancho3.columns]

# Resetear el índice
df_ancho3.reset_index(inplace=True)

# Mostrar el DataFrame final


df_ancho3.columns = df_ancho3.columns.str.strip().str.lower() 
df_ancho3.columns = df_ancho3.columns.str.replace(' ', '_')


# Diccionario de mapeo para abreviaturas en inglés
abbreviations = {
    'azúcares_totales_(g)_valor_por_100g': 'total_sugars_100g',
    'colesterol_(mg)_valor_por_100g': 'cholesterol_100g',
    'energía_(kcal)_valor_por_100g': 'energy_100g',
    'fibra_(g)_valor_por_100g': 'fiber_100g',
    'grasas_monoinsaturadas_(g)_valor_por_100g': 'mono_fats_100g',
    'grasas_poliinsaturadas_(g)_valor_por_100g': 'poly_fats_100g',
    'fibra_(g)_valor_por_porción': 'fiber_serving',
    'grasas_monoinsaturadas_(g)_valor_por_porción': 'mono_fats_serving',
    'grasas_poliinsaturadas_(g)_valor_por_porción': 'poly_fats_serving',
    'grasas_saturadas_(g)_valor_por_porción': 'sat_fats_serving',
    'grasas_totales_(g)_valor_por_porción': 'total_fats_serving',
    'grasas_trans_(g)_valor_por_porción': 'trans_fats_serving',
    'hidratos_de_carbono_disponibles_(g)_valor_por_porción': 'available_carbs_serving',
    'proteínas_(g)_valor_por_porción': 'proteins_serving',
    'sodio_(mg)_valor_por_porción': 'sodium_serving',
}


df_ancho3.rename(columns=abbreviations, inplace=True)

df_ancho3['tamaño'] = df_ancho3['nombre_producto'].str.extract(r'(\d+(?:\.\d+)?\s*[a-zA-Z]+)$')


def separate_size(size):
    match = re.match(r'(\d+(?:\.\d+)?)\s*(L|ml|g|kg|cc)', size)
    if match:
        value, unit = match.groups()
        return pd.Series([value, unit])  # Devolver como serie
    return pd.Series([None, None])  # Si no hay coincidencia

# Convert 'tamaño' column to string type
df_ancho3['tamaño'] = df_ancho3['tamaño'].astype(str)

# Aplicar la función y crear nuevas columnas para Valor y Unidad
df_ancho3[['valor', 'unidad']] = df_ancho3['tamaño'].apply(separate_size)

df_ancho3.drop(columns=['tamaño'], inplace=True)

df_ancho3['marca'] = df_ancho3['marca'].replace('Frutas Y Verduras Propias', 'Jumbo')
df_ancho3['precio'] = df_ancho3['precio'].str.replace('$', '', regex=False)
df_ancho3['precio'] = df_ancho3['precio'].str.replace('.', '', regex=False)
df_ancho3['precio'] = df_ancho3['precio'].replace('', '0').astype(int)  # Replace empty strings with '0' before converting to int
df_ancho3['codigo_producto'] = df_ancho3['codigo_producto'].str.replace('Código: ', '', regex=False)

df_ancho3.fillna(0, inplace=True)

cols_to_convert = [
    "total_sugars_100g", "cholesterol_100g", "energy_100g", "fiber_100g",
    "mono_fats_100g", "poly_fats_100g", "grasas_saturadas_(g)_valor_por_100g",
    "grasas_totales_(g)_valor_por_100g", "grasas_trans_(g)_valor_por_100g",
    "hidratos_de_carbono_disponibles_(g)_valor_por_100g", "proteínas_(g)_valor_por_100g",
    "sodio_(mg)_valor_por_100g", "azúcares_totales_(g)_valor_por_porción",
    "colesterol_(mg)_valor_por_porción", "energía_(kcal)_valor_por_porción",
    "fiber_serving", "mono_fats_serving", "poly_fats_serving", "sat_fats_serving",
    "total_fats_serving", "trans_fats_serving", "available_carbs_serving",
    "proteins_serving", "sodium_serving"
]

# Convertir cada columna a tipo float
# Verificar las columnas existentes en df_ancho2
existing_columns = df_ancho3.columns

# Filtrar cols_to_convert para incluir solo las columnas que existen en df_ancho2
cols_to_convert = [col for col in cols_to_convert if col in existing_columns]

for col in cols_to_convert:
    df_ancho3[col] = df_ancho3[col].astype(str).str.replace(r'[^\d]', '', regex=True)  
    df_ancho3[col] = pd.to_numeric(df_ancho3[col])  # Convierte a float

# Confirmar las conversiones


df_ancho3 = df_ancho3[df_ancho3['precio'] != 0]

df_ancho3.head()

# histograma para la columna 'precio'






,nombre_producto,precio,codigo_producto,marca,ingredientes,azãºcares_totales_(g)_valor_por_100g,energã­a_(kcal)_valor_por_100g,fiber_100g,grasas_totales_(g)_valor_por_100g,hidratos_de_carbono_disponibles_(g)_valor_por_100g,...,sodio_(mg)_valor_por_100g,azãºcares_totales_(g)_valor_por_porcion,energã­a_(kcal)_valor_por_porcion,fibra_(g)_valor_por_porcion,grasas_totales_(g)_valor_por_porcion,hidratos_de_carbono_disponibles_(g)_valor_por_porcion,proteã­nas_(g)_valor_por_porcion,sodio_(mg)_valor_por_porcion,valor,unidad
0,ChampiÃ±Ã³n Bandeja 200 g,1690,CÃ³digo: 1333653,Jumbo,ChampiÃ±Ã³n entero,0.5,29,0,6,9,...,0,1,58,0,1.2,1.8,9.4,0,200,g
1,Lechuga HidropÃ³nica EspaÃ±ola 1 un.,1590,CÃ³digo: 1463041,Pura Hoja,Lechuga espaÃ±ola hidropÃ³nica,0.94,13,11,22,223,...,5,0.5,6.5,0.6,0.1,1.1,0.7,2.5,0,0
2,LimÃ³n Malla 1 kg,1790,CÃ³digo: 260818,Jumbo,LimÃ³n,2.5,29,0,3,53,...,2,2.5,29,0,0.3,53,1,2,1,kg
